In [3]:
import pandas as pd
import numpy as np
import sys
import progeny
import scanpy as sc
from anndata import AnnData

In [4]:
#!mkdir data
#!wget http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz -O data/pbmc3k_filtered_gene_bc_matrices.tar.gz
#!cd data; tar -xzf pbmc3k_filtered_gene_bc_matrices.tar.gz
#!mkdir write

--2021-02-04 17:38:50--  http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz
Resolving cf.10xgenomics.com (cf.10xgenomics.com)... 2606:4700::6812:1ad, 2606:4700::6812:ad, 104.18.0.173, ...
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|2606:4700::6812:1ad|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz [following]
--2021-02-04 17:38:50--  https://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|2606:4700::6812:1ad|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7621991 (7,3M) [application/x-tar]
Saving to: ‘data/pbmc3k_filtered_gene_bc_matrices.tar.gz’

data/pbmc3k_filtere 100%[===================>]   7,27M  5,83MB/s    in 1,2s    

2021-02-04 17:38:52 (5,83 MB/s) - ‘data/pbmc3k_fi

In [ ]:
model <- getModel(organism, top=top)
common_genes <- intersect(rownames(expr), rownames(model))

    if (verbose){
        number_genes <- apply(model, 2, function (x) {
            sum(rownames(model)[which (x != 0)] %in% unique(rownames(expr)))
        })
        message("Number of genes used per pathway to compute progeny scores:")
        message(paste(names(number_genes),": ", number_genes, " (", 
            (number_genes/top)*100,"%)",sep = "","\n"))
    }
    
    if (perm==1) {
        result <- t(expr[common_genes,,drop=FALSE]) %*% 
            as.matrix(model[common_genes,,drop=FALSE])
        
        if (scale && nrow(result) > 1) {
            rn <- rownames(result)
            result <- apply(result, 2, scale)
            rownames(result) <- rn
        }
    
    } else if (perm > 1) {
      expr <- data.frame(names = row.names(expr), row.names = NULL, expr)
      model <- data.frame(names = row.names(model), row.names = NULL, model)
      result <- progenyPerm(expr, model, k = perm, z_scores = z_scores, 
          get_nulldist = get_nulldist)
    }

In [14]:
adata = sc.read_10x_mtx(
    'data/filtered_gene_bc_matrices/hg19/',
    var_names='gene_symbols',                
    cache=True)                              
adata.var_names_make_unique()

# Basic filtering
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

# Annotate the group of mitochondrial genes as 'mt'
adata.var['mt'] = adata.var_names.str.startswith('MT-')  
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

# Filter cells following standard QC criteria.
adata = adata[adata.obs.n_genes_by_counts < 2500, :]
adata = adata[adata.obs.pct_counts_mt < 5, :]

# Normalize the data
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# Identify the 2000 most highly variable genes
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)#min_mean=0.0125, max_mean=3, min_disp=0.5)

# Filter higly variable genes
adata.raw = adata
adata = adata[:, adata.var.highly_variable]

# Regress and scale the data
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(adata, max_value=10)

/home/badi/anaconda3/envs/ml/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:138: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [52]:
data = adata
if isinstance(data, AnnData):
    if data.raw is None:
        data = pd.DataFrame(np.transpose(data.X), index=data.var.index, 
                               columns=data.obs.index)
    else:
        data = pd.DataFrame(np.transpose(data.raw.X.toarray()), index=data.raw.var.index, 
                               columns=data.raw.obs_names)

model = progeny.getModel()
common_genes = np.array(model.index.intersection(data.index).to_list())
np.array(data.loc[common_genes].T).dot(np.array(model.loc[common_genes,]))

array([[ 26.71283212,   3.85237927,  10.75376961, ...,  -3.07260884,
          6.21726507,  45.71789235],
       [ 41.0373794 ,   4.05017319,   5.97691955, ...,   1.78053667,
          1.18154783,  16.1403584 ],
       [  2.67311364,   2.93957568, -20.31735917, ...,   0.        ,
          5.10106144,  49.99246117],
       ...,
       [ 20.21121114,  -9.72525334,   0.        , ...,   1.7503634 ,
          1.03472579,   6.71342246],
       [ 24.19041443,   5.54032715,  13.59175714, ...,  -6.46311173,
          9.21124538,  27.97518865],
       [ 27.992925  ,  -2.20573471,  42.29704002, ...,   0.        ,
          4.81708837,   0.        ]])